# Code to release particles in HYCOM, multifiles, Equator for 30 days, forward

In [1]:
# parcels
from parcels import VectorField, Field, SummedField, FieldSet, ParticleSet, Variable, ScipyParticle, JITParticle, AdvectionRK4, plotTrajectoriesFile, ParcelsRandom, ErrorCode

# open data to validate
from netCDF4 import Dataset
from glob import glob

# others
import numpy as np
import numpy.matlib
from datetime import timedelta, datetime
import xarray as xr
import os

# plotting
import matplotlib.pyplot as plt
from matplotlib import rc
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
%matplotlib inline

In [2]:
# create list of files; that's from 1_01 to 99_23, why not
dirin = '/projects2/rsmas/tidaldrift/hycom/'
uvfiles = dirin + '102_archv.2014_0*_*_uv_0+15m.nc'
sshfiles = dirin + '102_archs.2014_0*_*_ssh_0m.nc'

In [3]:
ds1 = xr.open_dataset(dirin+'102_archv.2014_001_01_uv_0+15m.nc')
ds1

<xarray.Dataset>
Dimensions:    (MT: 1, Depth: 2, Y: 7055, X: 9000)
Coordinates:
  * MT         (MT) datetime64[ns] 2014-01-01T01:00:28.800000
    Date       (MT) float64 ...
  * Depth      (Depth) float32 0.0 15.0
  * Y          (Y) int32 1 2 3 4 5 6 7 8 ... 7049 7050 7051 7052 7053 7054 7055
  * X          (X) int32 1 2 3 4 5 6 7 8 ... 8994 8995 8996 8997 8998 8999 9000
    Latitude   (Y, X) float32 ...
    Longitude  (Y, X) float32 ...
Data variables:
    u          (MT, Depth, Y, X) float32 ...
    bot_u      (MT, Y, X) float32 ...
    v          (MT, Depth, Y, X) float32 ...
    bot_v      (MT, Y, X) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM
    source:       HYCOM archive file
    experiment:   10.2
    history:      archv2ncdf3z

In [4]:
ds2 = xr.open_dataset(dirin+'102_archs.2014_001_01_ssh_0m.nc')
ds2

<xarray.Dataset>
Dimensions:        (MT: 1, Y: 7055, X: 9000)
Coordinates:
  * MT             (MT) datetime64[ns] 2014-01-01T01:00:00
    Date           (MT) float64 ...
  * Y              (Y) int32 1 2 3 4 5 6 7 ... 7050 7051 7052 7053 7054 7055
  * X              (X) int32 1 2 3 4 5 6 7 ... 8995 8996 8997 8998 8999 9000
    Latitude       (Y, X) float32 ...
    Longitude      (Y, X) float32 ...
Data variables:
    steric_ssh     (MT, Y, X) float32 ...
    nonsteric_ssh  (MT, Y, X) float32 ...
    ssh            (MT, Y, X) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM
    source:       HYCOM archive file
    experiment:   10.2
    history:      archv2ncdf2d

In [5]:
# create a dictionary 
filenames1 = {'U': uvfiles,
             'V': uvfiles}
filenames2 = {'SSH': sshfiles}

# map u, v, to the names in the velocity files
variables1 = {'U': 'u',
             'V': 'v'}
variables2 = {'SSH' : 'ssh'}

# map lon, lat, depth, time to the names in the velocity files
dimensions1 = {'depth':'Depth',
              'lon': 'Longitude',
              'lat': 'Latitude'
              }
dimensions2 = {'lon': 'Longitude',
              'lat': 'Latitude'
              }
# but is it going to be sorted?
display(filenames1)
display(filenames2)

{'U': '/projects2/rsmas/tidaldrift/hycom/102_archv.2014_0*_*_uv_0+15m.nc',
 'V': '/projects2/rsmas/tidaldrift/hycom/102_archv.2014_0*_*_uv_0+15m.nc'}

{'SSH': '/projects2/rsmas/tidaldrift/hycom/102_archs.2014_0*_*_ssh_0m.nc'}

In [6]:
# select one layer (0 for surface, 1 for 15m deep)
indices = {'depth': [0]}

In [7]:
# not needed, see below?
# timestamps = np.expand_dims(np.array([np.datetime64('2014-01-01T%.2d:00:00' %m) 
#                                      for m in range(1,24)]), axis=1)
#timestamps = np.expand_dims(np.array(np.datetime64('2014-01-01T01:00:00')),axis=0)
timestamps = np.expand_dims(np.arange('2014-01-01T01', '2014-04-10T00', dtype='datetime64[h]'),axis=1)

In [8]:
print(len(filenames2))
print(len(timestamps))

1
2375


In [9]:
# define the first "Fieldset" from velocity files
fieldset = FieldSet.from_netcdf(filenames1, variables1, dimensions1, indices, mesh='spherical',
                                 allow_time_extrapolation=False,deferred_load=True,timestamps=timestamps)
# fieldset = FieldSet.from_netcdf(filenames, variables, dimensions, indices, mesh='spherical',
#                                  allow_time_extrapolation=False,deferred_load=True)

In [10]:
filenames2.keys()

dict_keys(['SSH'])

In [11]:
# create an extra field for SSH
sshfieldset = FieldSet.from_netcdf(filenames2,variables2,dimensions2,mesh='spherical',
                                 allow_time_extrapolation=False,deferred_load=True,timestamps=timestamps)

In [12]:
fieldset.add_field(sshfieldset.SSH)

In [13]:
display(fieldset.U.dimensions)
display(fieldset.U.indices)
display(fieldset.SSH.dimensions)
display(fieldset.SSH.indices)

{'depth': 'Depth', 'lon': 'Longitude', 'lat': 'Latitude'}

{'depth': [0], 'lon': range(0, 9000), 'lat': range(0, 7055)}

{'lon': 'Longitude', 'lat': 'Latitude'}

{'lon': range(0, 9000), 'lat': range(0, 7055), 'depth': [0]}

In [14]:
# define a new field, X, which will be used to implement cyclic boundary conditions
X = numpy.matlib.repmat(np.arange(1,9001,dtype=float), 7055, 1)
X = np.expand_dims(X, axis=0)
X = np.expand_dims(X, axis=0)
display(np.shape(X))
display(X[-1,-1])

(1, 1, 7055, 9000)

array([[1.000e+00, 2.000e+00, 3.000e+00, ..., 8.998e+03, 8.999e+03,
        9.000e+03],
       [1.000e+00, 2.000e+00, 3.000e+00, ..., 8.998e+03, 8.999e+03,
        9.000e+03],
       [1.000e+00, 2.000e+00, 3.000e+00, ..., 8.998e+03, 8.999e+03,
        9.000e+03],
       ...,
       [1.000e+00, 2.000e+00, 3.000e+00, ..., 8.998e+03, 8.999e+03,
        9.000e+03],
       [1.000e+00, 2.000e+00, 3.000e+00, ..., 8.998e+03, 8.999e+03,
        9.000e+03],
       [1.000e+00, 2.000e+00, 3.000e+00, ..., 8.998e+03, 8.999e+03,
        9.000e+03]])

In [15]:
fieldset.SSH.depth

array([0.], dtype=float32)

In [16]:
F = Field('X',X,lon=fieldset.U.lon,lat=fieldset.U.lat,depth=fieldset.SSH.depth,allow_time_extrapolation=True,
          time=np.ones(1) * timedelta(seconds=int(30*24*60*60)).total_seconds())
#F = Field('X',X,lon=fieldset.SSH.lon,lat=fieldset.SSH.lat)

In [17]:
fieldset.add_field(F)

In [18]:
# why none for X?
display(fieldset.X.dimensions)
display(fieldset.U.dimensions)
display(fieldset.SSH.dimensions)


None

{'depth': 'Depth', 'lon': 'Longitude', 'lat': 'Latitude'}

{'lon': 'Longitude', 'lat': 'Latitude'}

In [19]:
display(fieldset.SSH.data.data_shape)
display(fieldset.U.data.data_shape)
display(fieldset.X.data.shape)

(2375, 1, 7055, 9000)

(2375, 1, 7055, 9000)

(1, 7055, 9000)

In [20]:
print(fieldset.time_origin)

print(fieldset.U.dimensions)
print(fieldset.U.indices)
print(fieldset.U.creation_log)
print(fieldset.U.grid.time)

print(fieldset.SSH.dimensions)
print(fieldset.SSH.indices)
print(fieldset.SSH.creation_log)
print(fieldset.SSH.grid.time)

print(fieldset.X.dimensions)
print(fieldset.X.indices)
print(fieldset.X.creation_log)
print(fieldset.X.grid.time)

2014-01-01T01
{'depth': 'Depth', 'lon': 'Longitude', 'lat': 'Latitude'}
{'depth': [0], 'lon': range(0, 9000), 'lat': range(0, 7055)}
from_netcdf
[0.0000e+00 3.6000e+03 7.2000e+03 ... 8.5392e+06 8.5428e+06 8.5464e+06]
{'lon': 'Longitude', 'lat': 'Latitude'}
{'lon': range(0, 9000), 'lat': range(0, 7055), 'depth': [0]}
from_netcdf
[0.0000e+00 3.6000e+03 7.2000e+03 ... 8.5392e+06 8.5428e+06 8.5464e+06]
None
None

[2592000.]


In [21]:
fieldset.X.dimensions = fieldset.U.dimensions
fieldset.X.indices = fieldset.U.indices

In [22]:
print(fieldset.U.dimensions)
print(fieldset.U.indices)
print(fieldset.SSH.dimensions)
print(fieldset.SSH.indices)
print(fieldset.X.dimensions)
print(fieldset.X.indices)
print(fieldset.X.interp_method)

{'depth': 'Depth', 'lon': 'Longitude', 'lat': 'Latitude'}
{'depth': [0], 'lon': range(0, 9000), 'lat': range(0, 7055)}
{'lon': 'Longitude', 'lat': 'Latitude'}
{'lon': range(0, 9000), 'lat': range(0, 7055), 'depth': [0]}
{'depth': 'Depth', 'lon': 'Longitude', 'lat': 'Latitude'}
{'depth': [0], 'lon': range(0, 9000), 'lat': range(0, 7055)}
linear


In [23]:
# create a particle set along equator
t0 = 30*24*60*60 # time of the release [s]
plon = np.arange(-180,180)
plat = np.ones_like(plon) * 0
ptime = np.ones(len(plon)) * timedelta(seconds=int(t0)).total_seconds()
len(ptime)

360

In [24]:
# time can be set with the time= parameters
# but by default the particle will be release at
# the first time of the fieldset
pset = ParticleSet.from_list(fieldset=fieldset,         # fieldset
                             pclass=JITParticle,        # type of particles
                             #repeatdt=adv_reinject_dt, # relaunch freq
                             lon=plon,                  # release lon
                             lat=plat,                  # release lat
                             time=ptime)                # release time

In [25]:
# pset.show(show_time=0,domain={'N':20, 'S':-20, 'E':179, 'W':360-179})

In [26]:
display(pset[0],pset[-1])

P[0](lon=-180.000000, lat=0.000000, depth=0.000000, time=2592000.000000)

P[359](lon=179.000000, lat=0.000000, depth=0.000000, time=2592000.000000)

In [27]:
class SampleParticleInitZero(JITParticle):            # Define a new particle class
    ssh = Variable('SSH', initial=0)                  # Variable 'SSH' initially zero
    #u = Variable('U', initial=0)                      # Variable 'U','V' initially zero
    #v = Variable('V', initial=0)                      # Variable 'U','V' initially zero    
    x = Variable('X',initial=0)

In [28]:
pset = ParticleSet(fieldset=fieldset, pclass=SampleParticleInitZero, lon=plon, lat=plat, time=ptime)
display(pset[0],pset[-1])

P[360](lon=-180.000000, lat=0.000000, depth=0.000000, SSH=0.000000, X=0.000000, time=2592000.000000)

P[719](lon=179.000000, lat=0.000000, depth=0.000000, SSH=0.000000, X=0.000000, time=2592000.000000)

In [29]:
def Sample(particle, fieldset, time):
         particle.SSH = fieldset.SSH[time, particle.depth, particle.lat, particle.lon]
         particle.X = fieldset.X[time,particle.depth, particle.lat, particle.lon]        
         #particle.U = fieldset.U[time, particle.depth, particle.lat, particle.lon]
         #particle.V = fieldset.V[time, particle.depth, particle.lat, particle.lon]    
sample_kernel = pset.Kernel(Sample)    # Casting the Sample function to a kernel.

In [30]:
# function to delete particle when out of bounds
def DeleteParticle(particle, fieldset, time):
    particle.delete()

In [31]:
# below fails but works if I remove above the X sampling. 
pset.execute(sample_kernel, dt=0)

sh: None: command not found
INFO: Compiled ArraySampleParticleInitZeroSample ==> /tmp/parcels-3477/libd137f9d782228f71cd8b257c60d7d7f0_0.so


Correct cell not found for (-180.000000, 0.000000) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 0 0
            new particle indices: (yi, xi) 3467 0
            Mesh 2d shape:  7055 9000
            Relative particle position:  (xsi, eta) -6.3672140672782871e+03 1.0000000000000000e+00
Correct cell not found for (-179.000000, 0.000000) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 0 0
            new particle indices: (yi, xi) 3467 0
            Mesh 2d shape:  7055 9000
            Relative particle position:  (xsi, eta) -6.3421620795107037e+03 1.0000000000000000e+00
Correct cell not found for (-178.000000, 0.000000) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 0 0
            new particle indices: (yi, xi) 3467 0
            Mesh 2d shape:  7055 9000
            Relative particle position:  (xsi, eta) -6.3171100917431195e+03 1.0000000000000000e+00
Correct cell not found for (-177.000000, 0.000000) after 1000000 itera

OutOfBoundsError: 0
Particle P[360](lon=-180.000000, lat=0.000000, depth=0.000000, SSH=0.000000, X=0.000000, time=2592000.000000)
Time: 2014-01-31T01:00:00,	timestep dt: -0.000000
Out-of-bounds sampling by particle at (-180.000000, 0.000000, 0.000000)

In [ ]:
fieldset.X.grid.time

In [ ]:
#units? m per nanoseconds?
plt.plot(pset.V)

In [ ]:
# advection parameters
runtime = timedelta(days=1)                     # advection integration time
dt = timedelta(minutes=20)                       # advection timestep: what should it be compared to time step of fields (1h?)
#dt = timedelta(hours=1)                       # advection timestep: what should it be compared to time step of fields (1h?)
outputdt = timedelta(hours=1)                    # output frequency
outfile = 'test_to_delete.nc'
# output files for the trajectories
output_file = pset.ParticleFile(name=outfile, outputdt=outputdt)

In [ ]:
# del kernel
kernel1 = pset.Kernel(AdvectionRK4)

In [ ]:
# do not forget to set this up before!export CC=gcc on macOS

In [ ]:
# execute the integration
%time
pset.execute(kernel1 + sample_kernel,# + kernel2 + sample_kernel,          # the kernels (define how particles move)
     runtime=runtime,                         # the total length of the run
     dt=dt,                                   # the timestep of the kernel
     recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
     output_file=output_file)

In [ ]:
#os.system("parcels_convert_npydir_to_netcdf out-YYSVJCKA")
output_file.export()